<center><h1>HW4: NoSQL on Cloud</h1></center>

To enable to work on cloud, set <b>GOOGLE_APPLICATION_CREDENTIALS</b> with the value as your absolute path to your credentials JSON file and delete Your <b>BIGTABLE_EMULATOR_HOST</b> environment variable and restart your notebook, see [HERE](https://cloud.google.com/bigtable/docs/) for more. 
```bash
unset BIGTABLE_EMULATOR_HOST
export GOOGLE_APPLICATION_CREDENTIALS="/Users/Administrator1/study/CMU/23fall14848/homework/hw4-NoSQLonCloud/cloud-infrastructure-cluster-2a979a96afbf.json"
```

In [10]:
!unset BIGTABLE_EMULATOR_HOST
!unset GOOGLE_APPLICATION_CREDENTIALS
!export GOOGLE_APPLICATION_CREDENTIALS="/Users/Administrator1/study/CMU/23fall14848/homework/hw4-NoSQLonCloud/cloud-infrastructure-cluster-2a979a96afbf.json"

<h2>Install Required Packages</h2>

In [31]:
!pip install google-cloud-bigtable
!pip install google-cloud-happybase

  Using cached protobuf-4.24.4-cp37-abi3-macosx_10_9_universal2.whl (409 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.8.0
    Uninstalling protobuf-3.8.0:
      Successfully uninstalled protobuf-3.8.0


<h2>Initialize the Application</h2>

In [11]:
from google.cloud import bigtable
from google.cloud import happybase
from google.cloud.bigtable import column_family

#Populate project_id and instance_id if you are running on the cloud
project_id = "cloud-infrastructure-cluster"
instance_id = "bigtablehw4"

client = bigtable.Client(project=project_id, admin=True)
instance = client.instance(instance_id)


<h2>Create Tables</h2>

Create the table for the data of our sensor.  
Note that the data contains three different versions of sensor we used to capture different weather related information.  

Create the column family for all the weather related information readings.  

Hint: an efficient way to optimize your data storage starts with grouping and dividing your dataset into a set of columns.   
It would better for us to store the columns "Model_Used" and the "Notes" as a same column family "sensortype", and store "Humidity_Percentage", "Temperature_in_Fahrenheit", "Wind_Speed_in_mph", "Wind_Direction" as in a same family "weatherinfo"

Hint: to delete the table: 
```
print("Deleting the {} table.".format(table_id))
table.delete()
```

Hint: The GC policies for column family is for garbage collection. It restricts that there shouldn't be too much data of different timestamp versions of one cell. So GC makes sure how column families would automatically clear the "garbage".  

In [19]:
table_id = 'sensor'
print("Creating the {} table.".format(table_id))
table = instance.table(table_id)

print("Creating column family readingsection with Max Version GC rule...")
# Create a column family with GC policy : most recent N versions
# Define the GC policy to retain only the most recent 2 versions
max_versions_rule = column_family.MaxVersionsGCRule(2)
column_family_ids = ["sensortype", "weatherinfo"]
column_families = {cf_id: max_versions_rule for cf_id in column_family_ids} 
if not table.exists():
    table.create(column_families=column_families)
else:
    print("Table {} already exists.".format(table_id))
    # print("Deleting the {} table.".format(table_id))
    # table.delete()


Creating the sensor table.
Creating column family readingsection with Max Version GC rule...
Table sensor already exists.


<h2>Insert Rows into Tables</h2>

Insert the data readings into this created table.  
Use Pandas library for Python to read the data from the CSVs.  

The data source is "sersor.csv".  

Hint: don't store any N/A values in the database table.

Solution should be scalable to handle as many records as possible as long as they belong to one of the sensor models identified earlier. So I added `if row['Model_Used'] in MODELS:` to check if the model_used in future records are one of the models we identified ("modnist-sensor", "mednist-sensor", "oldnist-sensor")

In [18]:
!pip install pandas

In [20]:
import pandas as pd

file_path = 'sensors.csv'

# Step 1: Read CSV into DataFrame
df = pd.read_csv(file_path)
print("Writing some reading types to the table.")
df.head(5)

Writing some reading types to the table.


,Sensor_ID,Model_Used,Notes,Humidity_Percentage,Temperature_in_Fahrenheit,Wind_Speed_in_mph,Wind_Direction
0,AAAAAA,modnist-sensor,newer type of sensor,50%,45 F,4 mph,NW
1,AAAAAB,mednist-sensor,our mid-age quality sensor,40%,NaN,5 mph,SW
2,AAAAAC,oldnist-sensor,our oldest sensor,80%,NaN,NaN,NaN
3,AAAAAD,modnist-sensor,newer type of sensor,90%,95 F,2 mph,NE
4,AAAAAE,mednist-sensor,our mid-age quality sensor,40%,NaN,7 mph,SW


In [21]:
import datetime

# Define column families
sensor_type_family = "sensortype"
weather_info_family = "weatherinfo"

# Hardcoded column headers and model names
SENSOR_COLUMNS = ["Model_Used", "Notes"]
WEATHER_COLUMNS = ["Humidity_Percentage", "Temperature_in_Fahrenheit", "Wind_Speed_in_mph", "Wind_Direction"]
MODELS = ["modnist-sensor", "mednist-sensor", "oldnist-sensor"]

rows = []

# 2. Iterate over DataFrame rows
for _, row in df.iterrows():
    # Given that Sensor_ID is unique, 
    # it's appropriate to use it as the row key
    row_key = row['Sensor_ID'].encode()

    # Check if model is one of the known models; according to the requirment
    if row['Model_Used'] in MODELS:
        # initialize a row with this key
        bigtable_row = table.direct_row(row_key)
        
        # Setting cells for 'sensortype' column family
        for col in SENSOR_COLUMNS:
            # Checking if the value is not NaN and is not 'N/A'
            if pd.notna(row[col]) and row[col] != 'N/A': 
                bigtable_row.set_cell(
                    sensor_type_family,
                    col.encode(),
                    row[col].encode(),
                    timestamp=datetime.datetime.utcnow()
                )

        # Setting cells for 'weatherinfo' column family
        for col in WEATHER_COLUMNS:
            # Checking if the value is not NaN and is not 'N/A'
            if pd.notna(row[col]) and row[col] != 'N/A':
                bigtable_row.set_cell(
                    weather_info_family,
                    col.encode(),
                    row[col].encode(),
                    timestamp=datetime.datetime.utcnow()
                )
        rows.append(bigtable_row)
# insert rows
# 3. Write the constructed rows to Bigtable
table.mutate_rows(rows)

[, , , , , , , , , , , , , ]

<h2>Find a single Element in the Table</h2>

Hint: The `row_filter` in Bigtable can be used to apply certain conditions when querying rows. For instance, if we want to limit the number of cells returned per column or only want to see cells from a certain time range, `row_filter` can be handy.

<h2>Retrieve All Rows in BigTable Table!</h2>

In [27]:
print("Scanning for all sensors:")
partial_rows = table.read_rows()

# Loop through all the rows fetched
for row in partial_rows:
    # Print the key for each row (which represents the Sensor_ID)
    print(f"Reading data for {row.row_key.decode('utf-8')}:")
    
    # Loop through each column family and their columns in the row
    for column_family_id, columns in row.cells.items():
        for column, cell_list in columns.items():
            # Get the most recent cell from the list of cells
            cell = cell_list[0]
            # Print the column name and its value along with timestamp
            print(f"{column.decode('utf-8')}: {cell.value.decode('utf-8')} @{cell.timestamp}")
    print()  # Print a newline for better separation between rows


Scanning for all sensors:
Reading data for AAAAAA:
Model_Used: modnist-sensor @2023-10-13 01:22:55.395000+00:00
Notes: newer type of sensor  @2023-10-13 01:22:55.396000+00:00
Humidity_Percentage: 50% @2023-10-13 01:22:55.396000+00:00
Temperature_in_Fahrenheit: 45 F @2023-10-13 01:22:55.396000+00:00
Wind_Direction: NW @2023-10-13 01:22:55.396000+00:00
Wind_Speed_in_mph: 4 mph @2023-10-13 01:22:55.396000+00:00

Reading data for AAAAAB:
Model_Used: mednist-sensor @2023-10-13 01:22:55.397000+00:00
Notes: our mid-age quality sensor @2023-10-13 01:22:55.397000+00:00
Humidity_Percentage: 40% @2023-10-13 01:22:55.397000+00:00
Wind_Direction: SW @2023-10-13 01:22:55.397000+00:00
Wind_Speed_in_mph: 5 mph @2023-10-13 01:22:55.397000+00:00

Reading data for AAAAAC:
Model_Used: oldnist-sensor @2023-10-13 01:22:55.397000+00:00
Notes: our oldest sensor @2023-10-13 01:22:55.397000+00:00
Humidity_Percentage: 80% @2023-10-13 01:22:55.397000+00:00

Reading data for AAAAAD:
Model_Used: modnist-sensor @202

<h2>Delete Tables</h2>

In [ ]:
print("Deleting the {} table.".format(table_id))
table.delete()

<h3>Read the Example for More Inforamtion <a href="https://cloud.google.com/bigtable/docs/samples-python-hello">https://cloud.google.com/bigtable/docs/samples-python-hello</a></h3>